# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [21]:
import os
from dotenv import load_dotenv
config_list = [
  
    {
        'model': 'gpt4',
        'api_key': ''+(os.environ.get('AZURE_OPENAI_KEY'))+'',
        'base_url':  ''+(os.environ.get('AZURE_OPENAI_ENDPOINT'))+'',
        'api_type': 'azure',
        'api_version': '2024-02-01'
    }
]

In [22]:
llm_config={"model": "gpt-3.5-turbo"}

In [23]:
from autogen import ConversableAgent

## Creating the needed agents

In [24]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config={
        "config_list": config_list,
    },
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [25]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
      llm_config={
        "config_list": config_list,
    },
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [26]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
      llm_config={
        "config_list": config_list,
    },
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [27]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [28]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [32]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

Message:
Hello, I'm here to help you get started with our product.Could you tell me your name and location?

Carryover: 


********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

anurag from jersey city

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Nice to meet you, Anurag from Jersey City! That's all the information we need for now. 
TERMINATE

--------------------------------------------------------------------------------
customer_proxy_agent (to

## Print out the summary

In [33]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "name": "Anurag",
  "location": "Jersey City"
}


The assistant is interested in reading about science fiction.


The assistant kept the conversation light and engaging by incorporating local details related to Anurag's location of Jersey City. The assistant also shared a science fiction-themed joke and a short science fiction story, adding humor and intrigue into the conversation. However, it's not clear what Anurag's reading interests are, as the assistant didn't ask about them.




## Print out the cost

In [34]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

({'total_cost': 0.03504, 'gpt-4-32k': {'cost': 0.03504, 'prompt_tokens': 406, 'completion_tokens': 89, 'total_tokens': 495}}, {'total_cost': 0.03504, 'gpt-4-32k': {'cost': 0.03504, 'prompt_tokens': 406, 'completion_tokens': 89, 'total_tokens': 495}})


({'total_cost': 0.014519999999999998, 'gpt-4-32k': {'cost': 0.014519999999999998, 'prompt_tokens': 196, 'completion_tokens': 23, 'total_tokens': 219}}, {'total_cost': 0.014519999999999998, 'gpt-4-32k': {'cost': 0.014519999999999998, 'prompt_tokens': 196, 'completion_tokens': 23, 'total_tokens': 219}})


({'total_cost': 0.10511999999999999, 'gpt-4-32k': {'cost': 0.10511999999999999, 'prompt_tokens': 812, 'completion_tokens': 470, 'total_tokens': 1282}}, {'total_cost': 0.10511999999999999, 'gpt-4-32k': {'cost': 0.10511999999999999, 'prompt_tokens': 812, 'completion_tokens': 470, 'total_tokens': 1282}})


